In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/NLP-NER-V3.0.0

In [ ]:
# !pip install transformers

In [ ]:
import sys
sys.path.append('./utils')

In [ ]:
import torch

from Model import Model
import utils.common as utils
from utils.DataProcess import Processor
from Config import config
from Trainer import Trainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.device = device
print(config.device)

In [ ]:
logger = utils.get_logger(config.dataset)
logger.info(config)
config.logger = logger

In [ ]:
logger.info("Loading Data")
processor = Processor(config)
train_data = utils.read_from_file(config.train_data_path)
dev_data = utils.read_from_file(config.dev_data_path)
train_loader, dev_loader = processor(train_data, config.train_params), processor(dev_data, config.dev_params)

updates_total = len(train_data) // config.batch_size * config.epochs

In [ ]:
logger.info("Building Model")
model = Model(config)
model = model.to(config.device)

In [ ]:
trainer = Trainer(model, processor, config, updates_total)

best_f1 = 0
for i in range(1, config.epochs + 1):
    logger.info("Epoch: {}".format(i))
    trainer.train(i, train_loader)
    f1 = trainer.eval(i, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        trainer.save(config.save_path)     
logger.info("Best DEV F1: {:3.4f}".format(best_f1))

In [ ]:
trainer.load(config.save_path)
test_data = utils.read_from_file(config.test_data_path)
test_loader = processor(test_data)
trainer.predict("Final", test_loader, test_data)